<a href="https://colab.research.google.com/github/minoxd/fiber_optic_box_inspection/blob/main/hehe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environmental Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
import os
import shutil
import zipfile
import tarfile
import json
from sklearn.model_selection import train_test_split
import yaml
import torch
from multiprocessing import Pool
import pandas as pd

if os.path.exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

These files are required in the `Data_INF` folder:
- data_label_All.tar.gz
- inf_raw_classes_v1.0.json
- matching_name_id_v1.0.xlsx

In [5]:
BASE_DIR = '/content/drive/MyDrive/fiber_optic_box_inspection'
if not os.path.exists(BASE_DIR):
    raise FileNotFoundError(f"Base directory {BASE_DIR} not found. Check Google Drive mount.")

LOCAL_BASE_DIR = '/content'

paths = {
    'base_dir':                                   BASE_DIR,
    'Data_INF':                                   os.path.join(BASE_DIR, 'Data_INF'),
    'dataset_dir':                                os.path.join(BASE_DIR, 'dataset'),
    'yolo_output_dir':                            os.path.join(BASE_DIR, 'runs_yolo'),
}

if not os.path.exists(paths['dataset_dir']):
    os.makedirs(paths['dataset_dir'], exist_ok=True)

paths['raw_classes'] =                            os.path.join(paths['Data_INF'], 'inf_raw_classes_v1.0.json')
paths['new_raw_classes'] =                        os.path.join(paths['Data_INF'], 'inf_raw_classes_v1.0_new.json')
paths['matching_name_id'] =                       os.path.join(paths['Data_INF'], 'matching_name_id_v1.0.xlsx')
paths['data_label_compressed'] =                  os.path.join(paths['Data_INF'], 'data_label_ALL.tar.gz')

paths['processed_data_label_compressed'] =        os.path.join(paths['dataset_dir'], 'data_label_ALL_processed.tar.gz')

local_paths = {
    'base_dir':                                   LOCAL_BASE_DIR,
    'dataset_dir':                                os.path.join(LOCAL_BASE_DIR, 'dataset'),
}

if not os.path.exists(local_paths['dataset_dir']):
    os.makedirs(local_paths['dataset_dir'], exist_ok=True)

local_paths['data_label_dir'] =                   os.path.join(local_paths['dataset_dir'], 'data_label_v2_ALL')
local_paths['label_files_backup_dir'] =           os.path.join(local_paths['dataset_dir'], 'label_files_backup') #todo labels_backup
local_paths['processed_data_label_compressed'] =  os.path.join(local_paths['dataset_dir'], 'data_label_ALL_processed.tar.gz')

local_paths['data'] =                             os.path.join(local_paths['dataset_dir'], 'data.yaml')
local_paths['all_paths'] =                        os.path.join(local_paths['dataset_dir'], 'all.txt')
local_paths['train_paths'] =                      os.path.join(local_paths['dataset_dir'], 'train.txt')
local_paths['val_paths'] =                        os.path.join(local_paths['dataset_dir'], 'val.txt')
local_paths['test_paths'] =                       os.path.join(local_paths['dataset_dir'], 'test.txt')
local_paths['ids_from_label_files'] =             os.path.join(local_paths['dataset_dir'], 'ids_from_label_files_backup_dir.txt')

In [6]:
# local_paths['images'] = os.path.join(local_paths['dataset_dir'], 'images')
# local_paths['labels'] = os.path.join(local_paths['dataset_dir'], 'labels')

# if not os.path.exists(local_paths['images']):
#     os.makedirs(local_paths['images'], exist_ok=True)
# if not os.path.exists(local_paths['labels']):
#     os.makedirs(local_paths['labels'], exist_ok=True)

In [7]:
# # todo this needed where?
# def read_image_paths(file_path):
#     if os.path.exists(file_path):
#         with open(file_path, "r") as f:
#             return [line.strip() for line in f if line.strip()]
#     return []

# all_images = read_image_paths(paths['all_paths'])
# train_images = read_image_paths(paths['train_paths'])
# val_images = read_image_paths(paths['val_paths'])
# test_images = read_image_paths(paths['test_paths'])

## Get processed dataset

In [8]:
shutil.copytree(paths['dataset_dir'], local_paths['dataset_dir'], dirs_exist_ok=True)

'/content/dataset'

In [9]:
with tarfile.open(paths['processed_data_label_compressed'], 'r:gz') as tar_file:
    tar_file.extractall(local_paths['dataset_dir'])
    print(f"Extracted {paths['processed_data_label_compressed']} to {local_paths['dataset_dir']}")

Extracted /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_ALL_processed.tar.gz to /content/dataset


## learn custom class names todo

In [10]:
# matching_name_id_df

In [11]:
# matching_name_id_df = pd.read_excel(paths['matching_name_id'], sheet_name=0)
# matching_name_id = dict(zip(matching_name_id_df['Label ID'], matching_name_id_df['Tên Label']))
# print(matching_name_id[1.0])

In [12]:
# id_to_names = matching_name_id_df.groupby(matching_name_id_df['Label ID'].astype(int))['Tên Label'].apply(list).to_dict()

# def find_names_by_id(integer_part):
#     return id_to_names.get(int(integer_part), ["ID not found"])

# integer_part = 0
# names = find_names_by_id(integer_part)
# print(f"Names for IDs with integer part {integer_part}: {names}")

In [13]:
# id_to_names

# Data Preparation (Run once per lifetime)

## Extract compressed dataset to VM

The compressed dataset is located on GDrive. We extract the file directly from GDrive to VM's storage

In [ ]:
shutil.rmtree(local_paths['data_label_dir'])

In [ ]:
with tarfile.open(paths['data_label_compressed'], 'r:gz') as tar_file:
    tar_file.extractall(local_paths['dataset_dir'])
    print(f"Extracted {paths['data_label_compressed']} to {local_paths['dataset_dir']}")

Extracted /content/drive/MyDrive/fiber_optic_box_inspection/Data_INF/data_label_ALL.tar.gz to /content/dataset


In [ ]:
for root, dirs, files in os.walk(local_paths['data_label_dir']):
    print(len(files))

22452


## Split dataset & save dataset division

In [ ]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

all_images = [os.path.join(local_paths['data_label_dir'], f) for f in os.listdir(local_paths['data_label_dir']) if f.endswith('.jpg')]

train_images, temp_images = train_test_split(all_images, train_size=train_ratio, random_state=42)
val_images, test_images = train_test_split(temp_images, train_size=(val_ratio / (val_ratio + test_ratio)), random_state=42)

In [ ]:
with open(local_paths['all_paths'], 'w') as f:
    for img in all_images:
        f.write(f"{img}\n")

with open(local_paths['train_paths'], 'w') as f:
    for img in train_images:
        f.write(f"{img}\n")

with open(local_paths['val_paths'], 'w') as f:
    for img in val_images:
        f.write(f"{img}\n")

with open(local_paths['test_paths'], 'w') as f:
    for img in test_images:
        f.write(f"{img}\n")

## Inspect class ids from label files and json

### Get class ids from json

In [ ]:
with open(paths['raw_classes'], 'r') as f:
    class_dict = json.load(f)
print("Class dictionary from JSON:", class_dict)

class_ids_from_json = set(sorted([int(k) for k in class_dict.keys()]))

Class dictionary from JSON: {'0': '1.0', '1': '1.1', '2': '1.2', '4': '1.7', '7': '2.0', '8': '2.1', '9': '2.3', '10': '3.0', '11': '3.1', '12': '4.0', '13': '4.1', '14': '5.0', '15': '5.1', '16': '6.0', '17': '6.1', '18': '7.0', '19': '7.1', '20': '7.2', '21': '7.3', '23': '7.7', '24': '8.0', '25': '8.3', '27': '8.5', '28': '9.0', '30': '9.4', '32': '10.0', '33': '10.1', '34': '11.0', '35': '11.1', '36': '13.0', '37': '13.1', '38': '13.2', '41': '14.0', '42': '14.1', '43': '14.2', '44': '14.3', '45': '15.0', '46': '15.1', '47': '15.4', '50': '17.0', '51': '17.1', '52': '18.0', '53': '18.1', '54': '19.0', '55': '19.1', '56': '19.2', '57': '19.3', '58': '19.4', '59': '19.5', '60': '20.0', '61': '20.1', '62': '20.2', '63': '20.3', '67': '23.0', '68': '23.1', '69': '23.2', '70': '24.0', '71': '24.1', '72': '24.3', '73': '26.0', '74': '26.1', '75': '27.0', '76': '27.1', '77': '28.0', '78': '28.3', '80': '29.0', '81': '29.1', '82': '29.2', '83': '29.4', '84': '30.0', '85': '30.1', '86': '32

### Backup ids from label files before removal

In [ ]:
def get_class_ids_from_label_file(label_file):
    class_ids = set()
    msg = None
    try:
        ids = set()
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    id = line.strip().split()[0]
                    ids.add(id)
        msg = f'Ids in {label_file}: {ids}'
        class_ids.update(ids)
    except Exception as e:
        msg = f'Error processing {label_file}: {e}'
    return class_ids, msg

def get_class_ids_from_label_files(label_dir):
    label_files = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.txt')]
    with Pool() as pool:
        results = pool.map(get_class_ids_from_label_file, label_files)
    class_ids, msgs = zip(*results)
    class_ids = set().union(*class_ids)

    with open(local_paths['ids_from_label_files'], 'w') as f:
        f.write(str(len(msgs)) + '\n')
        for msg in msgs:
            f.write(f'{msg}\n')

    return set(sorted([int(id) for id in class_ids]))

In [ ]:
class_ids_from_label_files = get_class_ids_from_label_files(local_paths['data_label_dir'])

### Get class ids differences

In [ ]:
class_ids = class_ids_from_json & class_ids_from_label_files
class_ids_from_label_files_not_in_json = class_ids_from_label_files - class_ids
class_ids_from_json_not_in_label_files = class_ids_from_json - class_ids

In [ ]:
print(f'Class IDs from json (count: {len(class_ids_from_json)}):                                          \n\t {class_ids_from_json}')
print(f'Class IDs from label files (count: {len(class_ids_from_label_files)}):                            \n\t {class_ids_from_label_files}')
print(f'Class Ids from label files but not in json (count: {len(class_ids_from_label_files_not_in_json)}):\n\t {class_ids_from_label_files_not_in_json}')
print(f'Class Ids from json but not in label files (count: {len(class_ids_from_json_not_in_label_files)}):\n\t {class_ids_from_json_not_in_label_files}')

Class IDs from json (count: 151):                                          
	 {0, 1, 2, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 27, 28, 30, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 93, 94, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172}
Class IDs from label files (count: 150):                            
	 {0, 1, 2, 3, 4, 7, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 54, 55, 56, 57, 58, 60, 61, 62, 63,

### From label files, remove labels that their ids does not appear in json

In [ ]:
def process_label_file(args):
    label_file, label_dir, backup_dir, class_ids_to_remove = args
    filename = os.path.basename(label_file)
    try:
        with open(label_file, 'r') as f:
            lines = f.readlines()

        lines_kept = [
            line for line in lines
            if line.strip() and line.strip().split()[0] not in class_ids_to_remove
        ]

        lines_removed = len(lines) - len(lines_kept)

        if lines_removed <= 0:
            return

        backup_file = os.path.join(backup_dir, filename)
        shutil.copy(label_file, backup_file)
        with open(label_file, 'w') as f:
            f.writelines(lines_kept)

        return f"Processed {filename}: {lines_removed} lines removed, original backed up"
    except Exception as e:
        return f"Error processing {filename}: {e}"

def process_label_files(label_dir, backup_dir, class_ids_to_remove):
    class_ids_to_remove = [str(id) for id in class_ids_to_remove]

    all_labels = [
        os.path.join(label_dir, f) for f in os.listdir(label_dir)
        if f.endswith('.txt')
    ]

    args = [(f, label_dir, backup_dir, class_ids_to_remove) for f in all_labels]

    with Pool() as pool:
        results = pool.map(process_label_file, args)

    for result in results:
        if result:
            print(result)

    return len(all_labels)

In [ ]:
if not os.path.exists(local_paths['label_files_backup_dir']):
    os.makedirs(local_paths['label_files_backup_dir'])

num_files_processed = process_label_files(
    label_dir=local_paths['data_label_dir'],
    backup_dir=local_paths['label_files_backup_dir'],
    class_ids_to_remove=class_ids_from_label_files_not_in_json
)
print(f"Total files processed: {num_files_processed}")

Processed 2022_11_03_08_42_50-DNIP106463HO.txt: 1 lines removed, original backed up
Processed 2022_11_15_16_13_30-HNIP297028HO.txt: 2 lines removed, original backed up
Processed 2022_11_14_08_53_27-KTMP021131HO.txt: 1 lines removed, original backed up
Processed 2022_11_19_11_13_47-DNGP128199HO.txt: 1 lines removed, original backed up
Processed 2022_11_09_16_22_25-HNIP386031HO.txt: 1 lines removed, original backed up
Processed 2022_11_17_10_35_33-LDGP039070HO.txt: 1 lines removed, original backed up
Processed 2022_11_05_08_39_39-QNIP028100HO.txt: 1 lines removed, original backed up
Processed 2022_11_02_14_39_28-GLIP032281HO.txt: 1 lines removed, original backed up
Processed 2022_11_14_09_26_25-LANP020052HO.txt: 1 lines removed, original backed up
Processed 2022_11_19_16_47_34-TGGP031212HO.txt: 1 lines removed, original backed up
Processed 2022_11_01_15_36_53-CTOP020125HO.txt: 1 lines removed, original backed up
Processed 2022_11_02_09_20_19-GLIP035003HO.txt: 1 lines removed, original ba

### From json, remove ids that are not in label files, then save to a new json

In [ ]:
with open(paths['raw_classes'], 'r') as f:
    new_class_dict = json.load(f)

for id in class_ids_from_json_not_in_label_files:
    new_class_dict.pop(str(id))
try:
    with open(paths['new_raw_classes'], 'w') as f:
        json.dump(new_class_dict, f, indent=2)
    print(f"JSON file created: {paths['new_raw_classes']}")
except Exception as e:
    print(f"Error writing JSON file: {e}")

JSON file created: /content/drive/MyDrive/fiber_optic_box_inspection/Data_INF/inf_raw_classes_v1.0_new.json


## Write dataset configuration

In [ ]:
class_names = [new_class_dict[str(id)] if id in class_ids else '' for id in range(list(class_ids)[-1] + 1)]
print("Class names in order:", class_names)

Class names in order: ['1.0', '1.1', '1.2', '', '1.7', '', '', '2.0', '', '2.3', '3.0', '3.1', '', '', '5.0', '5.1', '6.0', '6.1', '7.0', '7.1', '7.2', '7.3', '', '7.7', '8.0', '8.3', '', '8.5', '9.0', '', '9.4', '', '10.0', '10.1', '11.0', '11.1', '13.0', '13.1', '13.2', '', '', '14.0', '14.1', '14.2', '14.3', '', '', '', '', '', '', '', '', '', '19.0', '19.1', '19.2', '19.3', '19.4', '', '20.0', '20.1', '20.2', '20.3', '', '', '', '23.0', '23.1', '23.2', '24.0', '24.1', '24.3', '26.0', '26.1', '27.0', '27.1', '28.0', '', '', '29.0', '29.1', '29.2', '29.4', '30.0', '30.1', '32.0', '32.1', '32.2', '', '', '', '', '35.0', '35.1', '', '', '37.0', '37.1', '37.3', '37.4', '38.0', '38.1', '39.0', '39.1', '40.0', '40.1', '40.2', '41.0', '41.1', '41.2', '42.0', '42.1', '43.0', '43.1', '44.0', '44.1', '44.2', '45.0', '45.1', '45.2', '', '46.0', '46.1', '46.3', '46.4', '47.0', '47.1', '47.3', '48.0', '48.2', '49.0', '49.3', '49.4', '50.0', '50.1', '50.2', '51.0', '51.2', '51.3', '52.0', '52.3',

In [ ]:
data = {
    "train": local_paths['train_paths'],
    "val": local_paths['val_paths'],
    "test": local_paths['test_paths'],
    "nc": len(class_names),
    "names": class_names
}

with open(local_paths['data'], "w") as file:
    yaml.dump(data, file, default_flow_style=False)

print("data.yaml has been generated!")

data.yaml has been generated!


## Backup processed files and configurations

In [ ]:
def compress_folders_to_tar_gz(folders, output_tar_gz):
    try:
        with tarfile.open(output_tar_gz, "w:gz") as tar:
            for folder in folders:
                if not os.path.isdir(folder):
                    print(f"Error: '{folder}' is not a directory or does not exist.")
                    continue
                folder_name = os.path.basename(folder)
                tar.add(folder, arcname=folder_name)
                print(f"Added folder: {folder}")
        print(f"Created archive: {output_tar_gz}")
        return True
    except Exception as e:
        print(f"Error creating archive: {e}")
        return False

In [ ]:
folders_to_compress = [local_paths['data_label_dir'], local_paths['label_files_backup_dir']]
compress_folders_to_tar_gz(folders_to_compress, local_paths['processed_data_label_compressed'])

Added folder: /content/dataset/data_label_v2_ALL
Added folder: /content/dataset/label_files_backup
Created archive: /content/dataset/data_label_ALL_processed.tar.gz


True

In [ ]:
def copy_folder_exclude(src_dir, dst_dir, exclude_entries=None):
    if exclude_entries is None:
        exclude_entries = set()

    try:
        def ignore_func(directory, contents):
            return [entry for entry in contents if entry in exclude_entries]

        shutil.copytree(src_dir, dst_dir, ignore=ignore_func, dirs_exist_ok=True)
        print(f"Folder copied to: {dst_dir}")
        return True
    except FileNotFoundError:
        print(f"Error: Source directory '{src_dir}' not found.")
        return False
    except FileExistsError:
        print(f"Error: Destination directory '{dst_dir}' already exists and cannot be overwritten.")
        return False
    except Exception as e:
        print(f"Error copying folder: {e}")
        return False

In [ ]:
exclude = {
    os.path.splitext(os.path.basename(local_paths['data_label_dir']))[0],
    os.path.splitext(os.path.basename(local_paths['label_files_backup_dir']))[0]}
copy_folder_exclude(local_paths['dataset_dir'], paths['dataset_dir'], exclude)

Folder copied to: /content/drive/MyDrive/fiber_optic_box_inspection/dataset


True

## something else

In [ ]:
def get_distinct_filenames(directory):
    unique_names = set()
    try:
        for root, _, files in os.walk(directory):
            for f in files:
                name, _ = os.path.splitext(f)
                unique_names.add(name)

        return unique_names
    except Exception as e:
        print(f"Error scanning directory {directory}: {e}")
        return set()

distinct_names = get_distinct_filenames(local_paths['data_label_dir'])

In [ ]:
for name in distinct_names:
    image = f'{name}.jpg'
    label = f'{name}.txt'
    shutil.move(os.path.join(local_paths['data_label_dir'], image), local_paths['images'])
    shutil.move(os.path.join(local_paths['data_label_dir'], label), local_paths['labels'])

In [ ]:
with open(local_paths['all_paths'], 'r') as f:
    content = f.read()
new_content = content.replace('data_label_v2_ALL', 'images')
with open(local_paths['all_paths'], 'w') as f:
    f.write(new_content)
del content, new_content

with open(local_paths['train_paths'], 'r') as f:
    content = f.read()
new_content = content.replace('data_label_v2_ALL', 'images')
with open(local_paths['train_paths'], 'w') as f:
    f.write(new_content)
del content, new_content

with open(local_paths['val_paths'], 'r') as f:
    content = f.read()
new_content = content.replace('data_label_v2_ALL', 'images')
with open(local_paths['val_paths'], 'w') as f:
    f.write(new_content)
del content, new_content

with open(local_paths['test_paths'], 'r') as f:
    content = f.read()
new_content = content.replace('data_label_v2_ALL', 'images')
with open(local_paths['test_paths'], 'w') as f:
    f.write(new_content)
del content, new_content

# Model Training

## GPU check

In [14]:
!nvidia-smi

Mon Jun 23 11:38:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   41C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU is available! Using device: {device}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")


GPU is available! Using device: cuda
GPU Name: NVIDIA L4
Number of GPUs: 1
Total Memory: 23.80 GB


## Train

In [ ]:
model = YOLO('yolov8m.pt')
results = model.train(
    data=local_paths['data'],
    epochs=300,
    patience=10,
    batch=25,
    imgsz=896,
    save_period=5,
    cache='disk',
    device=0,
    workers=16,
    project=paths['yolo_output_dir'],
    name='v8m_mni1.0_batch25_imgsz896_',
    save_conf=True,)

100%|██████████| 49.7M/49.7M [00:01<00:00, 39.1MB/s]


Ultralytics 8.3.158 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=25, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=v8m_mni1.0_batch25_imgsz896_2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, po

100%|██████████| 755k/755k [00:00<00:00, 93.8MB/s]

Overriding model.yaml nc=80 with nc=173

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralyti

 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  2   4207104  ultralytics.nn.modules.block.C2f             [960, 576, 2]                 
 22        [15, 18, 21]  1   3875863  ultralytics.nn.modules.head.Detect           [173, [192, 384, 576]]        
Model summary: 169 layers, 25,956,487 parameters, 25,956,471 gradients, 79.6 GFLOPs

Transferred 469/475 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

     38/300        19G     0.9377     0.6334      1.189        164        896: 100%|██████████| 315/315 [05:39<00:00,  1.08s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:24<00:00,  1.37it/s]


                   all       1684      17375      0.663      0.562       0.56      0.375

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     39/300        19G     0.9386     0.6314      1.187        144        896: 100%|██████████| 315/315 [05:39<00:00,  1.08s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:25<00:00,  1.36it/s]


                   all       1684      17375       0.65      0.582      0.566      0.378

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     40/300      19.1G     0.9268     0.6169      1.177        514        896:  23%|██▎       | 71/315 [01:16<04:19,  1.06s/it]

## Resume Train

In [17]:
model = YOLO('/content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_mni1.0_batch25_imgsz896_2/weights/last.pt')
results = model.train(resume=True)

Ultralytics 8.3.158 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=25, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_mni1.0_batch25_imgsz896_2/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=v8m_mni1.0_batch25_imgsz896_2, nbs=64, nms=False, opset=None, 

100%|██████████| 755k/755k [00:00<00:00, 109MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

100%|██████████| 5.35M/5.35M [00:00<00:00, 306MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2618.2±972.5 MB/s, size: 101.7 KB)


...

     60/300      19.1G     0.8681     0.5612      1.146        103        896: 100%|██████████| 315/315 [05:42<00:00,  1.09s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:25<00:00,  1.36it/s]

                   all       1684      17375      0.668      0.575      0.579      0.384


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 50, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

12 epochs completed in 1.234 hours.
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_mni1.0_batch25_imgsz896_2/weights/last.pt, 52.2MB
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_mni1.0_batch25_imgsz896_2/weights/best.pt, 52.2MB

Validating /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_mni1.0_batch25_imgsz896_2/weights/best.pt...
Ultralytics 8.3.158 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 92 layers, 25,939,927 parameters, 0 gradients, 79.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:24<00:00,  1.40it/s]


                   all       1684      17375       0.64      0.557      0.581      0.385
                   1.0        163        164      0.816      0.866        0.9      0.781
                   1.1         67         67      0.679      0.694      0.745      0.595
                   1.2         16         16      0.267      0.438      0.334      0.187
                   1.7         20         20      0.381        0.5      0.518      0.414
                   2.0        270        272      0.931      0.926      0.971      0.862
                   2.3          1          1          1          0          0          0
                   3.0        251        251      0.891      0.932      0.946      0.607
                   3.1          4          5      0.551        0.2      0.223      0.126
                   5.0        267        527      0.923      0.991      0.981      0.693
                   5.1         10         10      0.824      0.472      0.539      0.261
                   6.

# what

In [ ]:
import torch
torch.cuda.empty_cache()